In [1]:
import numpy as np
from itertools import combinations

In [2]:
data = np.load("portfolio_qubo_data.npz")

Q = data['Q']
q = data['q']
mu = data['mu']
Sigma = data['Sigma']
B = int(data['B'])
TICKERS = list(data['TICKERS'])
n = len(TICKERS)

print("Data loaded successfully.")
print(f"n = {n} (assets)")
print(f"B = {B} (cardinality)")
print(f"Q shape: {Q.shape}")
print(f"q shape: {q.shape}")

Data loaded successfully.
n = 21 (assets)
B = 4 (cardinality)
Q shape: (21, 21)
q shape: (21,)


In [3]:
def f_qubo(x):
    return x @ Q @ x + q @ x

def metrics_for_indices(sel_idx):
    w = np.zeros(n)
    w[sel_idx] = 1.0 / B
    mu_day = float(mu @ w)
    var_day = float(w @ Sigma @ w)
    mu_ann = 252 * mu_day
    std_ann = np.sqrt(252 * var_day)
    return mu_ann, std_ann

In [4]:
best_val = np.inf
best_x = None

print("\nRunning brute-force search")
for comb in combinations(range(n), B):
    x = np.zeros(n)
    x[list(comb)] = 1
    val = f_qubo(x)
    if val < best_val:
        best_val = val
        best_x = x.copy()

sel_idx_BF = np.where(best_x == 1)[0]
mu_ann_BF, std_ann_BF = metrics_for_indices(sel_idx_BF)
sharpe_BF = mu_ann_BF / std_ann_BF


Running brute-force search


In [5]:
print("\n[Brute-Force Results Saved]")
print(f"Optimal f(x) = {best_val:.6f}")
print(f"Annualized μ = {mu_ann_BF:.6f}")
print(f"Annualized σ = {std_ann_BF:.6f}")
print(f"Sharpe Ratio = {sharpe_BF:.3f}")
print(f"Selected assets: {[TICKERS[i] for i in sel_idx_BF]}")


[Brute-Force Results Saved]
Optimal f(x) = -0.001198
Annualized μ = 0.477126
Annualized σ = 0.187264
Sharpe Ratio = 2.548
Selected assets: [np.str_('MSFT'), np.str_('JNJ'), np.str_('JPM'), np.str_('MS')]


In [6]:
np.savez(
    "bruteforce_results.npz",
    fx_bruteforce=np.array(best_val),
    mu_ann_bruteforce=np.array(mu_ann_BF),
    std_ann_bruteforce=np.array(std_ann_BF),
    sharpe_bruteforce=np.array(sharpe_BF),
    x_bruteforce=best_x
)